In [1]:
%%capture
!python -m pip install --upgrade pip

In [2]:
%%capture
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git 
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "deepseek-ai/deepseek-coder-7b-instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [5]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [6]:
print(model)#Printing the model to find the Query Key Values 

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaLinearScalingRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (n

In [7]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8, 
    lora_alpha=16, 
    target_modules=["q_proj", "k_proj", "v_proj"], #The same ase ["Query Key Values"]
    lora_dropout=0.05, 
    bias='none', 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 2097152 || all params: 3504607232 || trainable%: 0.059839858254335763


In [8]:
from datasets import load_dataset

data = load_dataset("RayBernard/leetcode")
data
#data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 2359
    })
})

In [9]:
tokenizer.pad_token = tokenizer.eos_token  # This is needed for models without a native pad token

# Define the function to process your data
def process_data(examples):
    # Tokenize the 'text' column
    tokenized_text = tokenizer(
        examples['text'], 
        truncation=True,  # Enable truncation to cut off at max length
        padding="max_length",  # Enable padding to ensure uniform sequence length
        max_length=512,  # Define max length according to your model's capacity
        return_tensors="pt"  # Return PyTorch tensors directly
    )
    return tokenized_text

# Apply the function to your dataset
tokenized_datasets = data.map(process_data, batched=True, remove_columns=data['train'].column_names)

Map:   0%|          | 0/2359 [00:00<?, ? examples/s]

In [13]:
import transformers

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_datasets["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        #eval_batch_size=1,
        seed=42,
        #total_train_batch_size=16,
        gradient_accumulation_steps=16,
        warmup_steps=2,
        max_steps=10,
        learning_rate=2e-5,
        fp16=True,
        logging_steps=1,
        output_dir="outputs2",
        optim="paged_adamw_8bit",
        remove_unused_columns=False,  # Make sure to keep this as needed
        num_train_epochs=2
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # Usually for inference performance optimization
trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
1,1.302800
2,1.347700
3,1.353100
4,1.344100
5,1.335400
6,1.347600
7,1.342100
8,1.362500
9,1.340300
10,1.361400


TrainOutput(global_step=10, training_loss=1.3437012076377868, metrics={'train_runtime': 164.8696, 'train_samples_per_second': 3.882, 'train_steps_per_second': 0.061, 'total_flos': 1.299675088945152e+16, 'train_loss': 1.3437012076377868, 'epoch': 0.2711864406779661})

In [17]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs2")

In [18]:
lora_config = LoraConfig.from_pretrained('outputs2')
model = get_peft_model(model, lora_config)

Expected output isn't met

In [19]:
text = "Generate a Leetcode twosum code in python"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Generate a Leetcode twosum code in python
Code in
Code in
Code in
Code in
Code in
Code in

